In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pymongo import MongoClient
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')  # Pour éviter les warnings inutiles

# Étape 1: Charger les données depuis MongoDB (plus cohérent avec le projet Docker)
# Note: Si tu lances ce notebook hors Docker, utilise "mongodb://localhost:27017/"
# Dans un container Jupyter, utilise "mongodb://mongo:27017/"
client = MongoClient("mongodb://localhost:27017/")
db = client["rice_db"]
collection = db["prices"]

# Récupère toutes les données (exclut _id)
df = pd.DataFrame(list(collection.find({}, {"_id": 0})))

# Fallback vers CSV si DB vide (pour tests isolés)
if df.empty:
    print("⚠️ DB vide, fallback vers CSV...")
    df = pd.read_csv("../data/prix_riz_madagascar.csv")
    print("Dataset chargé depuis CSV.")
else:
    print("✅ Dataset chargé depuis MongoDB.")

print(f"Dataset initial : {len(df)} lignes")

# Étape 2: Nettoyage et transformation des données (Pandas)
# Conversion des types, suppression NaN, tri chronologique
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Gère formats variés
df = df.dropna(subset=['date', 'region', 'type', 'price'])  # Supprime incomplets
df['price'] = pd.to_numeric(df['price'], errors='coerce')  # Assure float
df = df.sort_values('date').reset_index(drop=True)  # Tri chrono

# Ajout feature temporelle : numéro de jour depuis le début
df['date_num'] = (df['date'] - df['date'].min()).dt.days

print(f"Dataset nettoyé : {len(df)} lignes")
print(f"Colonnes : {df.columns.tolist()}")
print(f"Période : {df['date'].min().date()} à {df['date'].max().date()}")

# Étape 3: Statistiques descriptives (moyenne, min, max, tendance)
# Stats par région
stats_region = df.groupby("region")["price"].agg(['mean', 'min', 'max', 'count']).round(2)
print("\n📊 Prix par région (moyenne, min, max en Ar) :")
print(stats_region)

# Stats par type
stats_type = df.groupby("type")["price"].agg(['mean', 'min', 'max', 'count']).round(2)
print("\n📊 Prix par type :")
print(stats_type)

# Tendance mensuelle (groupe par mois)
df['month'] = df['date'].dt.to_period('M')
tendance_mensuelle = df.groupby('month')['price'].agg(['mean', 'count']).reset_index()
tendance_mensuelle['month'] = tendance_mensuelle['month'].astype(str)

print("\n📈 Tendance mensuelle (moyenne) :")
print(tendance_mensuelle.head(10))  # Top 10 pour ne pas polluer

# Visualisation : Tendance
plt.figure(figsize=(10, 5))
plt.plot(tendance_mensuelle['month'], tendance_mensuelle['mean'], marker='o', linewidth=2)
plt.title("Tendance Mensuelle des Prix Moyens du Riz à Madagascar")
plt.xlabel("Mois")
plt.ylabel("Prix Moyen (Ar)")
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Visualisation bonus : Boxplot par région
plt.figure(figsize=(10, 6))
df.boxplot(column='price', by='region', ax=plt.gca())
plt.title("Distribution des Prix par Région")
plt.suptitle("")  # Enlève le titre auto
plt.ylabel("Prix (Ar)")
plt.xticks(rotation=45)
plt.show()

# Étape 4: Prédiction simple (Régression Linéaire avec features)
# Features : date_num (numérique) + region/type (catégorielles via OneHot)
X = df[['date_num', 'region', 'type']]
y = df['price']

# Préprocessing : OneHot pour catégories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', ['date_num']),  # Garde date_num tel quel
        ('cat', OneHotEncoder(drop='first', sparse_output=False), ['region', 'type'])  # Drop first pour éviter multicolinéarité
    ]
)

# Pipeline complet
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Split train/test (80/20, stratifié par date pour séries temporelles)
# Pour plus de réalisme, utilise un split temporel : train sur passé, test sur futur
split_idx = int(len(df) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Fit et prédiction
X_train_pre = preprocessor.fit_transform(X_train)
X_test_pre = preprocessor.transform(X_test)

model = LinearRegression().fit(X_train_pre, y_train)
y_pred = model.predict(X_test_pre)

# Métriques
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"\n🔮 Modèle Régression Linéaire :")
print(f"Score R² (sur test) : {r2:.3f}")
print(f"RMSE (erreur moyenne) : {rmse:.2f} Ar")

# Visualisation prédictions vs réels
plt.figure(figsize=(8, 5))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Prix Réels (Ar)")
plt.ylabel("Prix Prédits (Ar)")
plt.title("Prédictions vs Réalité")
plt.grid(True, alpha=0.3)
plt.show()

# Étape 5: Prédiction future (ex. : +30 jours, pour une région/type donnée)
# Assume une région/type pour la prédiction (ex. : Antananarivo, blanc)
future_region = 'Antananarivo'  # Change ici
future_type = 'blanc'  # Change ici
future_date = df['date'].max() + pd.Timedelta(days=30)
future_date_num = (future_date - df['date'].min()).days

# Crée un échantillon future avec les mêmes catégories
future_df = pd.DataFrame({
    'date_num': [future_date_num],
    'region': [future_region],
    'type': [future_type]
})

# Transforme avec le preprocessor (fit sur train, transform sur future)
future_X_pre = preprocessor.transform(future_df)

prix_prevu = model.predict(future_X_pre)[0]
print(f"\n🌟 Prédiction pour {future_date.date()} ({future_region}, {future_type}) : {prix_prevu:.2f} Ar")

# Bonus : Intervalle de confiance simple (basé sur std des résidus)
residus_std = np.std(y_train - model.predict(X_train_pre))
intervalle = 1.96 * residus_std  # 95% approx
print(f"Intervalle 95% : [{prix_prevu - intervalle:.2f}, {prix_prevu + intervalle:.2f}] Ar")

# Fermeture client MongoDB
client.close()
print("\n✅ Analyse terminée ! Pour plus avancé : essaie ARIMA ou Prophet pour séries temporelles.")